In [1]:
import trimesh
import numpy as np
from collections import Counter
import time
import math
from tqdm import tqdm

In [2]:
#pick the mesh to process: 
file_location = "./test_meshes/"
file_location = "./"
file_name ="seperated_mesh.off"

In [3]:
def filter_mesh_significant_outside_pieces(unfiltered_mesh,significance_threshold=2000,n_sample_points=1000):
    """
    Purpose; will take in a full, unfiltered mesh and find the biggest mesh piece, and then return a list of that mesh 
    with all of the other mesh fragments that are both above the significance_threshold AND outside of the biggest mesh piece

    Pseudocode: 
    1) split the meshes to unconnected pieces
    2) Filter the meshes for only those above the significance_threshold
    3) find the biggest mesh piece
    4) Iterate through all of the remaining pieces:
        a. Determine if mesh inside or outside main mesh
        b. If outside add to final list to return

    Returns: 
    1) list of significant mesh pieces, including the main one that are not inside of main mesh

    """

    mesh_pieces = unfiltered_mesh.split(only_watertight=False)
    
    print(f"There were {len(mesh_pieces)} pieces after mesh split")

    significant_pieces = [m for m in mesh_pieces if len(m.faces) > significance_threshold]

    print(f"There were {len(significant_pieces)} pieces found after size threshold")
    if len(significant_pieces) <=0:
        print("THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold")
        return []

    #find piece with largest size
    max_index = 0
    max_face_len = len(significant_pieces[max_index].faces)

    for i in range(1,len(significant_pieces)):
        if max_face_len < len(significant_pieces[i].faces):
            max_index = i
            max_face_len = len(significant_pieces[i].faces)

    print("max_index = " + str(max_index))
    print("max_face_len = " + str(max_face_len))

    final_mesh_pieces = []

    main_mesh = significant_pieces[max_index]

    #final_mesh_pieces.append(main_mesh)
    for i,mesh in enumerate(significant_pieces):
        if i != max_index:
            #get a random sample of points
            # points = np.array(mesh.vertices[:n_sample_points,:]) # OLD WAY OF DOING THIS
            idx = np.random.randint(len(mesh.vertices), size=n_sample_points)
            points = mesh.vertices[idx,:]
            
            
            start_time = time.time()
            signed_distance = trimesh.proximity.signed_distance(main_mesh,points)
            print(f"Total time = {time.time() - start_time}")

            outside_percentage = sum(signed_distance < 0)/n_sample_points
            if outside_percentage > 0.9:
                final_mesh_pieces.append(mesh)
                print(f"Mesh piece {i} OUTSIDE mesh")
            else:
                print(f"Mesh piece {i} inside mesh :( ")
                
    return main_mesh,final_mesh_pieces



In [5]:
#start the global timer
global_timer = time.time()

unfiltered_mesh = trimesh.load_mesh(file_location + file_name)

#setting thresholds
significance_threshold=30 #number of faces needed for pieces to be considered to be kept
n_sample_points = 2 #number of points sampled on the mesh for determination of inside or outside
start_time = time.time()

#the main mesh is the first mesh in the piece
main_mesh,child_meshes = filter_mesh_significant_outside_pieces(unfiltered_mesh,
                            significance_threshold=significance_threshold,
                                n_sample_points=n_sample_points)
print(f"Total time for Mesh Cleansing: {time.time() - start_time}")


There were 3902 pieces after mesh split
There were 378 pieces found after size threshold
max_index = 0
max_face_len = 1845336
Total time = 24.6003897190094
Mesh piece 1 OUTSIDE mesh
Total time = 0.15836024284362793
Mesh piece 2 inside mesh :( 
Total time = 0.18016314506530762
Mesh piece 3 OUTSIDE mesh
Total time = 1.0256867408752441
Mesh piece 4 OUTSIDE mesh
Total time = 0.29471683502197266
Mesh piece 5 inside mesh :( 
Total time = 0.2189648151397705
Mesh piece 6 inside mesh :( 
Total time = 0.22297024726867676
Mesh piece 7 inside mesh :( 
Total time = 0.26123547554016113
Mesh piece 8 inside mesh :( 
Total time = 0.32075023651123047
Mesh piece 9 inside mesh :( 
Total time = 0.21797633171081543
Mesh piece 10 inside mesh :( 
Total time = 0.25991272926330566
Mesh piece 11 inside mesh :( 
Total time = 0.5279772281646729
Mesh piece 12 inside mesh :( 
Total time = 0.4516720771789551
Mesh piece 13 inside mesh :( 
Total time = 0.32538509368896484
Mesh piece 14 inside mesh :( 
Total time = 0.40

# checking who are axons or not

In [6]:
child_meshes_adding = main_mesh.copy()

faces_list = (np.ones(len(main_mesh.faces))).astype("int").tolist()

for jj,child_mesh_individual in enumerate(child_meshes):
    child_meshes_adding = child_meshes_adding + child_mesh_individual
    faces_list += (np.ones(len(child_mesh_individual.faces))*(jj+1)).astype("int").tolist()


len(faces_list)

2199363

In [ ]:
len(child_meshes_adding.faces)
print_trimesh(child_meshes_adding,"./test_meshes/child_and_main_mesh.off")
np.savez("./test_meshes/child_mesh_faces.npz",faces_list=faces_list)

In [ ]:
child_faces = np.load("./test_meshes/child_mesh_faces.npz")
child_faces["faces_list"]

In [40]:
child_meshes[35].show()

len(child_meshes[35].faces)
child_meshes[42].show()

In [38]:
axons_groups = [
    1,5,10,12,17,18,23,24,25,27,28,29,30,34,37,38,39,40,41
]

unsure = [19,20,22,21]

non_axon_group = [
    0,2,3,4,6,7,8,9,11,13,14,15,16,31,33,35,26,42
]

In [ ]:
std_dev_convex = np.std((trimesh.convex.adjacency_projections(submesh)))
if std_dev_convex < classifier_axon_std_dev_threshold:
mean_convex = abs(np.mean(trimesh.convex.adjacency_projections(submesh)))
                #print(f"total_faces  = {mean_convex}")
                if mean_convex > non_dendrite_convex_threshold:


In [52]:
#classifier_non_dendrite_convex_threshold (float) : Segment must be above this mean convex value to be considered a possible axon, cilia or error(default = 26.5) 
#classifier_axon_std_dev_threshold (float): standard deviation of convex measurements for which axon branches are under this threshold (default = 69.0) 
classifier_axon_std_dev_threshold = 69.0
classifier_non_dendrite_convex_threshold = 12

 
for i,ax in enumerate(axons_groups):
    current_mesh = child_meshes[ax]
    std_convex = np.std((trimesh.convex.adjacency_projections(current_mesh)))
    mean_convex = abs(np.mean(trimesh.convex.adjacency_projections(current_mesh)))
    print(f"{ax}: std_convex  = {std_convex}, mean_convex = {mean_convex}, num_faces = {len(current_mesh.faces)}")
    print(f"         convex_threshold  = {(std_convex < classifier_axon_std_dev_threshold)}, mean_threshold = {(mean_convex > classifier_non_dendrite_convex_threshold)}")
    

#the only ones that don't obey that rule is the really small ones don't follow the mean rule

1: std_convex  = 48.4240255897393, mean_convex = 12.457645468313999, num_faces = 16458
         convex_threshold  = True, mean_threshold = True
5: std_convex  = 46.294462245796666, mean_convex = 14.455115519208963, num_faces = 5608
         convex_threshold  = True, mean_threshold = True
10: std_convex  = 44.67788976117355, mean_convex = 9.936791841877042, num_faces = 3524
         convex_threshold  = True, mean_threshold = False
12: std_convex  = 46.323662254441125, mean_convex = 13.677988468752154, num_faces = 6432
         convex_threshold  = True, mean_threshold = True
17: std_convex  = 47.01093252049029, mean_convex = 20.807372684576098, num_faces = 348
         convex_threshold  = True, mean_threshold = True
18: std_convex  = 50.00154406836818, mean_convex = 21.707065194176323, num_faces = 268
         convex_threshold  = True, mean_threshold = True
23: std_convex  = 45.899212727683235, mean_convex = 18.58115673362776, num_faces = 1256
         convex_threshold  = True, mean_thre

In [53]:
for i,ax in enumerate(non_axon_group):
    current_mesh = child_meshes[ax]
    std_convex = np.std((trimesh.convex.adjacency_projections(current_mesh)))
    mean_convex = abs(np.mean(trimesh.convex.adjacency_projections(current_mesh)))
    print(f"{ax}: std_convex  = {std_convex}, mean_convex = {mean_convex}, num_faces = {len(current_mesh.faces)}")
    print(f"         convex_threshold  = {(std_convex < classifier_axon_std_dev_threshold)}, mean_threshold = {(mean_convex > classifier_non_dendrite_convex_threshold)}")
    

#the only ones that don't obey that rule is the really small ones don't follow the mean rule



0: std_convex  = 44.616039810724146, mean_convex = 10.190789955818586, num_faces = 117018
         convex_threshold  = True, mean_threshold = False
2: std_convex  = 42.11559274636262, mean_convex = 5.168460961383823, num_faces = 32560
         convex_threshold  = True, mean_threshold = False
3: std_convex  = 46.0326559699901, mean_convex = 12.573475117428487, num_faces = 27890
         convex_threshold  = True, mean_threshold = True
4: std_convex  = 44.22952270939172, mean_convex = 10.457317731661124, num_faces = 17314
         convex_threshold  = True, mean_threshold = False
6: std_convex  = 46.30503494275779, mean_convex = 11.354257034586862, num_faces = 16714
         convex_threshold  = True, mean_threshold = False
7: std_convex  = 44.34407522310204, mean_convex = 9.237599731207027, num_faces = 8374
         convex_threshold  = True, mean_threshold = False
8: std_convex  = 45.26274991177284, mean_convex = 11.516189191973027, num_faces = 5420
         convex_threshold  = True, mean_

In [54]:
"""
Final rule: Will do the easier facet finding for those meshes with 
mean_convex > 12
face_num less than 7000

THIS COULD CHANGE IF SEE THE OTHER DENDRITES ARE NOT BEING ADEQUATELY CONNECTED
"""

'\nFinal rule: Will do the easier facet finding for those meshes with \nmean_convex > 12\nface_num less than 7000\n\nTHIS COULD CHANGE IF SEE THE OTHER DENDRITES ARE NOT BEING ADEQUATELY CONNECTED\n'